In [1]:

import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, GlobalAveragePooling1D, Add
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

with open('../../dataset/train.json', 'r') as f:
    train_data = json.load(f)
with open('../../dataset/test.json', 'r') as f:
    test_data = json.load(f)

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)


train_embeddings = train_data['audio_embedding'].apply(np.array)
max_len = train_embeddings.apply(len).max()
embedding_dim = len(train_embeddings.iloc[0][0])

train_X = pad_sequences(train_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')
train_Y = train_data['is_turkey'].values

valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_embeddings = test_data.loc[valid_idx, 'audio_embedding'].apply(np.array)
test_X = pad_sequences(test_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')


B, T, D = train_X.shape
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X.reshape(-1, D)).reshape(B, T, D)
test_X = scaler.transform(test_X.reshape(-1, D)).reshape(test_X.shape[0], T, D)


def transformer_encoder(x, head_size, num_heads, ff_dim, dropout=0.1):

    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    attn_output = Dropout(dropout)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([x, attn_output]))

    ff = Dense(ff_dim, activation="relu")(out1)
    ff = Dropout(dropout)(ff)
    ff = Dense(x.shape[-1])(ff)
    out2 = LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))
    return out2


def build_large_transformer(seq_len, d_model):
    inputs = Input(shape=(seq_len, d_model))
    x = inputs

    for _ in range(2):  
        x = transformer_encoder(x, head_size=128, num_heads=16, ff_dim=256, dropout=0.2)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["AUC"])
    return model


good_models = []
for i in range(10):
    print(f"\n--- Training iteration {i+1}/200 ---")
    train_X_, val_X, train_Y_, val_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=45)

    model = build_large_transformer(seq_len=max_len, d_model=embedding_dim)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)

    model.fit(train_X_, train_Y_,
              validation_data=(val_X, val_Y),
              epochs=80,
              batch_size=32,
              callbacks=[early_stop],
              verbose=0)

    y_pred_prob = model.predict(val_X, verbose=0).ravel()
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision = precision_score(val_Y, y_pred)
    recall = recall_score(val_Y, y_pred)
    auc = roc_auc_score(val_Y, y_pred_prob)
    acc = accuracy_score(val_Y, y_pred)
    f1 = f1_score(val_Y, y_pred)

    if precision >= 0.97 and recall >= 0.96 and auc >= 0.95 and f1 >= 0.96 and acc >= 0.97:
        good_models.append({
            'model': model,
            'auc': auc,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

good_models = sorted(good_models, key=lambda x: -x['precision'])


if good_models:
    print("\n=== Top models with precision >= 0.95 and recall >= 0.95 ===")
    for i, m in enumerate(good_models):
        print(f"[Model {i+1}]")
        print(f"  Precision : {m['precision']:.4f}")
        print(f"  Recall    : {m['recall']:.4f}")
        print(f"  AUC       : {m['auc']:.4f}")
        print(f"  Accuracy  : {m['accuracy']:.4f}")
        print(f"  F1 Score  : {m['f1']:.4f}")
else:
    print("Không có mô hình nào đạt các tiêu chí >= 0.95.")


if good_models:
    best_model = good_models[0]['model']
    test_pred_prob = best_model.predict(test_X, verbose=0).ravel()
    test_data['is_turkey'] = -1.0
    test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob
    test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
    print("\n✅ Đã lưu kết quả dự đoán từ mô hình tốt nhất vào 'result.csv'")



--- Training iteration 1/200 ---

--- Training iteration 2/200 ---

--- Training iteration 3/200 ---

--- Training iteration 4/200 ---

--- Training iteration 5/200 ---

--- Training iteration 6/200 ---

--- Training iteration 7/200 ---

--- Training iteration 8/200 ---

--- Training iteration 9/200 ---

--- Training iteration 10/200 ---
Không có mô hình nào đạt các tiêu chí >= 0.95.


In [6]:
good_models

[]